In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Not sure where the unnamed column came from but just drop it
df = pd.read_csv('../data/processed_data.csv').drop(u'Unnamed: 0', axis=1) 
nan_indices = pd.isnull(df).any(1).nonzero()[0]
df_test = df.iloc[nan_indices]
df = df.dropna()
print(df.shape)
print(df.columns)
df.head()

(25697, 13)
Index([u'season', u'Opponent DRtg/A', u'shot_distance', u'left_side',
       u'middle', u'right_side', u'moving', u'playoffs', u'home', u'segababa',
       u'Dunk or Layup', u'shot_made_flag', u'total_seconds_remaining'],
      dtype='object')


,season,Opponent DRtg/A,shot_distance,left_side,middle,right_side,moving,playoffs,home,segababa,Dunk or Layup,shot_made_flag,total_seconds_remaining
0,1,108.06,256.0,0,0,1,0,0,1,0,0,0.0,2202
1,1,101.68,228.0,0,0,1,0,0,0,0,0,0.0,2048
2,1,109.65,323.0,0,0,1,0,0,0,1,0,1.0,1957
3,1,109.65,0.0,0,1,0,0,0,0,1,0,0.0,1834
4,1,109.65,148.0,0,1,0,0,0,0,1,0,1.0,1767


In [52]:
print (df_test.shape)
df_test.head()

(5000, 13)


,season,Opponent DRtg/A,shot_distance,left_side,middle,right_side,moving,playoffs,home,segababa,Dunk or Layup,shot_made_flag,total_seconds_remaining
5,1,108.57,102.0,0,1,0,0,0,0,0,0,NaN,1878
7,1,108.57,0.0,0,1,0,0,0,0,0,1,NaN,925
28,1,109.44,256.0,0,0,1,0,0,0,0,0,NaN,1811
29,1,109.44,15.0,1,0,0,0,0,0,0,0,NaN,1860
30,1,109.44,250.0,0,1,0,0,0,0,0,0,NaN,1538


In [53]:
y = df['shot_made_flag']
X = df.drop('shot_made_flag', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## XGBoost

In [54]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic'}

clf = GridSearchCV(
    xgb.XGBClassifier(**ind_params),
    cv_params,
    scoring = 'roc_auc', 
    cv=3, n_jobs=-1
)


clf.fit(X_train, y_train)

print "Done" # Takes awhile to run...


Done


In [57]:
print clf.cv_results_

{'std_train_score': array([  2.39661575e-03,   2.49559953e-03,   2.65139887e-03,
         5.15427059e-04,   1.03227103e-03,   1.73685822e-03,
         2.30255684e-05,   1.59537277e-04,   4.15303558e-04]), 'rank_test_score': array([3, 1, 2, 6, 5, 4, 7, 9, 8]), 'mean_score_time': array([ 0.05399998,  0.06699999,  0.06233335,  0.11199999,  0.0916667 ,
        0.10666664,  0.12100005,  0.11533324,  0.10466671]), 'param_max_depth': masked_array(data = [3 3 3 5 5 5 7 7 7],
             mask = [False False False False False False False False False],
       fill_value = ?)
, 'std_test_score': array([ 0.0055517 ,  0.00590843,  0.00664525,  0.00516236,  0.00589692,
        0.00507251,  0.00178153,  0.00246877,  0.00127594]), 'split1_train_score': array([ 0.82064181,  0.80997208,  0.8024155 ,  0.96960261,  0.95040367,
        0.9343317 ,  0.99981443,  0.99692743,  0.99014466]), 'split0_test_score': array([ 0.60833727,  0.6103696 ,  0.60761276,  0.5924181 ,  0.59192513,
        0.59631028,  0.5920

In [58]:
print clf.score(X_test, y_test)

0.628303948302
